# Nerdsniped by sequential decision problem

By [Andrew Wheeler](https://crimede-coder.com/)

So was nerdsniped a bit by this problem on [X](https://x.com/ChShersh/status/1935290942581301741). For a description if you cannot read X:

> Quant interview question:
> 
> You press a button that gives your randomly uniformly distributed number between $0 and $100K
> 
> Each time you press, you have two choices:
> 
> 1. Stop and take this amount of money
> 2. Try again
> 
> You can try 10 times total.
> When do you stop?

Part of the reason I was interested in many of the answers to the problem (including the person who posed the question), are saying it is a secretary problem, in which you view ~3 and then pick the max out of the prior you have seen. There is no need to view 3 cases to understand the distribution, you know it exactly. If you draw 100k you should take it, not wait to view future draws. Also the need to stop at 10 changes the problem as well.

![](Screenshot.PNG)

I debate on doing leet code style questions like this sometimes (mixing stats + coding). I don't think I would use this (I have asked expected value questions in prior data science interviews, and most everyone fails them), but here is how I thought about the problem. If you draw a value $X$, you can calculate the probability of getting a higher value in the subsequent $k$ draws. So say you got a value of 70k on the first draw. I am going to renormalize all the numbers between 0 and 1 to make it easier, so pretend here you got a draw of 0.7. The probability of drawing a higher value in the subsequent 9 draws is almost 96%.

$$1 - 0.7^9 \approx 0.96$$

So we should keep going. You can then just continue this logic for each draw. If the probability of getting a higher value is greater than 0.5, keep going. Here is a simulation to show the logic.

Make sure to read to the end, as following some of the X thread resulted in a better solution.

In [1]:
import numpy as np
np.random.seed(10)

# My initial attempt
def sim(prob=0.5,verbose=False,n=10000):
    x = 10 - np.arange(10) - 1
    regret = []
    picks = []
    for _ in range(n):
        sim = np.random.rand(10)
        ms = sim.max()
        for l,s in zip(x,sim):
            future_prob = 1 - s**l
            if future_prob < prob:
                pick_max = s
                break
            elif l == 0:
                pick_max = s
        r = ms - pick_max
        regret.append(r)
        picks.append(pick_max)
    picks = np.array(picks)
    regret = np.array(regret)
    if verbose:
        print(picks.mean())
        print(regret.mean())
        return None
    return picks, regret

sim(0.5,True)

0.8362183541670718
0.07173332896820378


One of the things I calculate here is the *regret* -- if you could pick perfectly (see all 10 draws and pick the max), what would be the expected return. Here it ends up being a smidge over 0.9 (the sum of `picks + regret`). So basically any strategy we think of, it will not beat making around 90k on average. (There is probably a way using extreme value distributions to figure out that distribution exactly as well.)

One of the things I noticed when doing this, changing the probability to not be 0.5 actually gets a higher expected value. If you use 0.8, our expected return is now around $85k.

In [2]:
sim(0.8,True) # setting the probability to 0.8 gets a higher expected value

0.85536420614866
0.055070965607778966


This is obviously sub-optimal. It would suggest if on draw 9 you had $30k, don't pick another one, because the probability of drawing higher is only 70%, not 80%. I mean it is not as risky (so should be smaller variance). But it does appear from the expected value, picking probabilities 0.6-0.8 results in higher expected returns to the game.

In [3]:
# Which this appers to do pretty well
t = np.linspace(0.5,0.95,10)

# tops out around 85k
for i in t:
    ev,rv = sim(i)
    print("")
    print(i,rv.mean(),ev.mean())


0.5 0.07212950486892737 0.8369544663643035

0.55 0.06318835405384854 0.8475709073970679

0.6 0.055547614077148616 0.8545588713154775

0.65 0.052528962521430876 0.8554965773521448

0.7 0.051492053917225396 0.857099694572431

0.75 0.052780829897087406 0.8554183591075935

0.8 0.05499302004432168 0.8544919460318391

0.85 0.06092134392256444 0.8482347458152982

0.8999999999999999 0.07119758580830286 0.8375272915329715

0.95 0.0930938415997185 0.8155751567364791


# A Better Solution via ChatGPT

[Cate Hall](https://x.com/catehall/status/1935438452788752514) made the comment that it is an error and make a joke that LLMs are better. John Horton on that thread showed how ChatGPT could answer the problem. I asked a few more [clarifying questions](https://chatgpt.com/share/68540ea4-d098-800e-9638-c329ceb441e4), and had python crack out the exact values. Instead of using probabilities, you can recursively see the expected value of future draws, and use that expected value to calculate whether to stop or continue on.

In [4]:
# Getting the exact values for the recurrence relationship
# I asked a few more clarifying questions to figure this out
# https://chatgpt.com/share/68540ea4-d098-800e-9638-c329ceb441e4
k = 10
ev = [0.5]

for i in range(k-1):
    lastev = ev[-1]
    leftover = (lastev + 1)/2
    newev = (lastev*lastev) + (1-lastev)*leftover
    ev.append(newev)

ev.reverse()
exact_rule = ev[1:] + [0]
exact_rule # I append 0 just so it is easier for the way I wrote my simulation

[0.8498214073624081,
 0.8364465402671089,
 0.8203006037631678,
 0.800375666500635,
 0.7750815008766949,
 0.741729736328125,
 0.6953125,
 0.625,
 0.5,
 0]

In [5]:
# Now simulating to see how this does
def sim_rule(rule=exact_rule,verbose=False,n=10000):
    x = 10 - np.arange(10) - 1
    regret = []
    picks = []
    for _ in range(n):
        sim = np.random.rand(10)
        ms = sim.max()
        for l,r,s in zip(x,rule,sim):
            if s > r:
                pick_max = s
                break
            elif l == 0:
                pick_max = s
        r = ms - pick_max
        regret.append(r)
        picks.append(pick_max)
    picks = np.array(picks)
    regret = np.array(regret)
    if verbose:
        print(picks.mean())
        print(regret.mean())
        return None
    return picks, regret

sim_rule(verbose=True) # ER is now at 86k

0.861607488905187
0.04759900635303128


And this verifies that ChatGPTs answer to the problem had a higher return than my strategy of using probabilities.